## Movie Recommendation Notebook
##### This workbook based on ideas collected all over mainly from the IBM Coursera course on Machine Learning to give movie recommendations based on the users preferences in movies.

#### Importing, Data Cleaning and Pre-processing

In [1]:
# importing the necessary packages
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# importing the datasets
movies_df = pd.read_csv('data/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# performing some cleaning on the data, removing the columns needed, etc.
# obtaining the date column from the title column and storing as it's own column
movies_df['year'] = movies_df.title.str.extract('(\(\d+\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d+)',expand=False)

# removing the years part from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d+\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\1200750126.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d+\))', '')


In [4]:
# since we will be using the user's selection for the recommendation system, we need the genres in their own dataframe
# to start we split the column data into a list
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
# creating a new dataframe since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

# iterating through the list of genres and one-hot encoding it
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

# filling all empty cells with 0 that does not have that genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### User Content Based Filtering System

In [6]:
# Given a user's random input of the movies they have liked and rated
inputMovies = pd.DataFrame([
            {'title':'Superman', 'rating':5},
            {'title':'Django Unchained', 'rating':3.5},
            {'title':'Godfather, The', 'rating':2},
            {'title':"V for Vendetta", 'rating':5},
            {'title':'Incredibles, The', 'rating':4.5},
            {'title':'Waiting to Exhale', 'rating':4},
            {'title':'If Lucy Fell', 'rating':5},
            {'title':'Desperado', 'rating':5},
            {'title':'Expendables, The', 'rating':5},
            {'title':'Apocalypto', 'rating':5}
         ])

In [7]:
# assigning the movie id to the titles in the inputMovies dataframe by matching titles
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\1812774450.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('year', 1)


,movieId,title,genres,rating
0,4,Waiting to Exhale,"[Comedy, Drama, Romance]",4.0
1,118,If Lucy Fell,"[Comedy, Romance]",5.0
2,163,Desperado,"[Action, Romance, Western]",5.0
3,858,"Godfather, The","[Crime, Drama]",2.0
4,2640,Superman,"[Action, Adventure, Sci-Fi]",5.0
5,140417,Superman,"[Action, Crime, Sci-Fi]",5.0
6,8961,"Incredibles, The","[Action, Adventure, Animation, Children, Comedy]",4.5
7,44191,V for Vendetta,"[Action, Sci-Fi, Thriller, IMAX]",5.0
8,48304,Apocalypto,"[Adventure, Drama, Thriller]",5.0
9,79695,"Expendables, The","[Action, Adventure, Thriller]",5.0


In [8]:
# since we have the movie IDs we can extract the users that have seen the same movie and reviewed them
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,118,If Lucy Fell,"[Comedy, Romance]",1996,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,163,Desperado,"[Action, Romance, Western]",1995,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
843,858,"Godfather, The","[Crime, Drama]",1972,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2556,2640,Superman,"[Action, Adventure, Sci-Fi]",1978,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8279,8961,"Incredibles, The","[Action, Adventure, Animation, Children, Comedy]",2004,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10896,44191,V for Vendetta,"[Action, Sci-Fi, Thriller, IMAX]",2006,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11352,48304,Apocalypto,"[Adventure, Drama, Thriller]",2006,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15696,79695,"Expendables, The","[Action, Adventure, Thriller]",2010,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20186,99114,Django Unchained,"[Action, Drama, Western]",2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
# dropping unnecessary columns not needed for this section
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\562384663.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\562384663.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\562384663.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\562384663.py:4: FutureWarning: In a future version of pa

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


##### Now, we would create a weighted average system for the movie by way of the genres

In [10]:
# obtain the product of the ratings and the genres to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             20.0
Animation              5.0
Children               5.0
Comedy                14.0
Fantasy                0.0
Romance               14.0
Drama                 16.0
Action                33.0
Crime                  5.5
Thriller              14.5
Horror                 0.0
Mystery                0.0
Sci-Fi                13.0
IMAX                   4.5
Documentary            0.0
War                    0.0
Musical                0.0
Western               10.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [11]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\quoej\AppData\Local\Temp\ipykernel_11340\789887408.py:4: FutureWarning: In a future version of pandas all arg

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.284790
2    0.161812
3    0.181230
4    0.284790
5    0.090615
dtype: float64

In [16]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head(10)

movieId
81132     0.731392
83266     0.721683
4956      0.721683
71999     0.715210
8481      0.695793
122787    0.666667
459       0.666667
64645     0.666667
75408     0.660194
115479    0.660194
dtype: float64

### Recommendation

In [18]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(10).keys())].reset_index(drop=True) 

,movieId,title,genres,year
0,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
1,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
2,8481,Northwest Passage,"[Action, Adventure, Drama, Romance, Thriller, ...",1940
3,64645,The Wrecking Crew,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1968
4,71999,Aelita: The Queen of Mars (Aelita),"[Action, Adventure, Drama, Fantasy, Romance, S...",1924
5,75408,Lupin III: Sweet Lost Night (Rupan Sansei: Swe...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2008
6,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
7,83266,Kaho Naa... Pyaar Hai,"[Action, Adventure, Comedy, Drama, Mystery, Ro...",2000
8,115479,"Whip Hand, The","[Action, Adventure, Crime, Drama, Sci-Fi, Thri...",1951
9,122787,The 39 Steps,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1959



## Original Author
Saeed Aghabozorgi

### Other Contributors
<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01" target="_blank">Joseph Santarcangelo</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-11-03        | 2.1     | Lakshmi    | Updated URL of csv                 |
| 2020-08-27        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |
|                   |         |            |                                    |
|                   |         |            |                                    |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
